# FYS-STK 4155 H20 PROJECT 1

## Definitions

In [87]:
# From the assignment; The function that we are goint to approximate
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

In [93]:
# Defining the approximated function's polinomial degree. NOTE: Deg x = deg y
polinomial_degree = 5

In [96]:
# Defining the space in which we will evaluate and approximate the FrankeFunction()
n = 4
x_start = y_start = 0.
x_end = y_end = 1.
x = np.linspace(x_start, x_end, n)
y = np.linspace(y_start, y_end, n)

In [89]:
# Adding some randn() noise to the FrankeFunction() and returning samples in an ndarray with just the right shape and ordered in step-wise increments of x for each full sweep of y
def noisy_franky(noise_sigma=0.):
    Z = np.ndarray([x.size, y.size])
    for i, xi in enumerate(x):
        Z[i,:] = FrankeFunction(xi, y) + noise_sigma*np.random.randn(n)
    return np.reshape(Z, x.size*y.size, order='F')

In [94]:
# Setting up the design matrix X. Here we must initialize it with the right shape of powers of x and y, reflecting the how we set up the sample array in noisy_franky() above
X = np.ndarray([n**2, 2*polinomial_degree+2])
for k in range(n):
    for i in range(polinomial_degree+1):
        X[k*n:(k+1)*n,i] = x[k]**i
    for i in range(polinomial_degree+1):
        X[k*n:(k+1)*n,polinomial_degree+1+i] = y**i